### **1. Template(Jinja2) Basics**
#### **1.1. Installing Jinja2**
To use Jinja2 with FastAPI, you need to install the `jinja2` package:
```bash
pip install jinja2==3.1.2
```

#### **1.2. Template Directory Setup**
FastAPI expects template files to reside in a specified directory (commonly named `templates`).

**Project Structure Example:**
```
my_project/
  ├── main.py
  └── templates/
      └── index.html
```
- **`main.py`**: Main FastAPI application file.
- **`templates/`**: Directory containing HTML template files.
- **`index.html`**: HTML file that defines the web page structure.

### Example HTML Code (`index.html`)
```html
<!DOCTYPE html>
<html>
<head>
    <title>FastAPI Template Example</title>
</head>
<body>
    <h1>Hello, {{ username }}</h1>
</body>
</html>
```
`{{ username }}` dynamically injects data from your Python code.

### **2. Jinja2Templates Class in FastAPI**
#### **2.1. Importing Jinja2Templates**
To enable template rendering, import the Jinja2Templates class:
```python
# my_project > main.py (1)
from fastapi import FastAPI, Request
from fastapi.templating import Jinja2Templates

app = FastAPI()
templates = Jinja2Templates(directory="templates")
```

#### **2.2. Creating a Route with TemplateResponse**
```python
# my_project > main.py (2)
@app.get("/")
def read_root(request: Request):
    return templates.TemplateResponse("index.html", {"request": request, "username": "William Kim"})
# uvicorn main:app --reload
```

<img src='../source/img/09/02/01.png' width='600px' height='300px'>

- **`TemplateResponse`**: Renders the specified template and passes dynamic data to the HTML.
- **`request`**: Required by FastAPI's TemplateResponse to include HTTP request data.

### **3. Dynamic Data Rendering with URL Parameters**
FastAPI can pass dynamic data to templates using **Path** and **Query Parameters**.

#### **3.1. Path Parameter Example**
```python
# my_project > main.py (3)
@app.get("/user/{username}")
def get_user(request: Request, username: str):
    return templates.TemplateResponse("index.html", {"request": request, "username": username})
# uvicorn main:app --reload
```

<img src='../source/img/09/03/01.png' width='800px' height='600px'>

- **URL Test**: Visiting `/user/woongkeol` renders `Hello, woongkeol` in the HTML page.

#### **3.2. Query Parameter Example**
```python
# my_project > main.py (4)
@app.get("/items/")
def read_items(request: Request, item_list: str = ""):
    items = item_list.split(",")
    return templates.TemplateResponse("index.html", {"request": request, "items": items})
# uvicorn main:app --reload
```
<img src='../source/img/09/03/02.png' width='800px' height='600px'>

- **URL Test**: Visiting `/items/?item_list=apple,samsung` dynamically renders the list in the HTML.

### **4. Jinja2 Filters for Data Handling**
#### **4.1. Safe Filter**
The `|safe` filter prevents HTML escaping, rendering HTML tags as actual HTML instead of plain text.

**Example Code:**
```python
# my_project > main.py (5) & index_with_safe.html
@app.get("/safe")
def read_root_safe(request: Request):
    my_variable_with_html = "<h1>Hello, FastAPI!</h1>"
    return templates.TemplateResponse("index_with_safe.html", {"request": request, "my_variable_with_html": my_variable_with_html})
# uvicorn main:app --reload
```

**HTML Template (`index_with_safe.html`)**
```html
{# index_with_safe.html #}
<!DOCTYPE html>
<html>
<head>
    <title>FastAPI Template Example</title>
</head>
<body>
    {{my_variable_with_html|safe}}
</body>
</html>
```

<img src='../source/img/09/04/01.png' width='800px' height='500px'>

> **Caution:** Be mindful when using `|safe` as it may expose your application to **XSS (Cross-Site Scripting)** attacks if user input is rendered without validation.

### 5. **Template Inheritance in Jinja2**
Template inheritance helps to define common layout structures like headers, footers, or navigation bars in a **base template** and extend it across multiple pages.

#### **5.1. Example Code Structure**
**`base.html`**
```html
{# template_project > base.html #}
<!DOCTYPE html>
<html>
<head>
    <title>My Website</title>
</head>
<body>
<header>
    <h1>Welcome to My Website!</h1>
</header>
{% block content %}{% endblock %}
</body>
</html>
```

**`index.html`**
```html
{# template_project > index.html #}
{% extends "base.html" %}
{% block content %}
<p>{{ text }}</p>
{% endblock %}
```

**Python Code Example:**
```python
# template_project > main.py
@app.get("/inherit")
def template_inherit(request: Request):
    return templates.TemplateResponse("index.html", {"request": request, "text": "FastAPI and Jinja2 Inheritance Example."})
# uvicorn main:app --reload
```

<img src='../source/img/09/05/01.png' width='1000px' height='500px'>

####  * * 
- **Enhanced Code Reusability:** Centralizes shared content to reduce duplication.
- **Improved Maintenance:** Updating content in the base template automatically reflects changes across all inherited templates.
- **Consistent Layout:** Ensures unified design across web pages.

### **6. Jinja2 Control Statements**
Jinja2 supports various control statements for conditional logic and loops.

#### **6.1. Conditional Statements**
- **If Statement:** `{% if ... %} ... {% endif %}`
- **Else Statement:** `{% else %}`
- **Elif Statement:** `{% elif ... %}`

**Example Code:**
```html
{# template_project > index.html (2) #}
{% if time_of_day == "morning" %}
<h1>Good Morning!</h1>
{% elif time_of_day == "afternoon" %}
<h1>Good Afternoon!</h1>
{% else %}
<h1>Good Evening!</h1>
{% endif %}
```

```python
# template_project > main.py (2)
@app.get("/")
def read_root(request: Request):
    current_hour = datetime.now().hour
    greet_by_time='evening'
    # If the hour is before 12, print "morning"
    if current_hour < 12:
        greet_by_time = "morning"
    # Else if the hour is before 18, print "afternoon"
    elif current_hour < 18:
        greet_by_time = "afternoon"
    # Otherwise, print "evening"
    else:
        greet_by_time = "evening"
    return templates.TemplateResponse("index.html", {"request": request, "time_of_day": greet_by_time})
# uvicorn main:app --reload
```

<img src='../source/img/09/06/01.png' width='700px' height='500px'>

#### **6.2. Loop Statements**
`{% for item in items %} ... {% endfor %}` renders each element in the provided list.

**Example Code:**
```html
{# template_project > index.html (3) #}
{% if items is defined and items is not none %}
    <ul>
    {% for item in items %}
        <li>{{ item }}</li>
    {% endfor %}
    </ul>
{% endif %}
</ul>
```

```python
# template_project > main.py (3)
@app.get("/loop")
def read_root(request: Request):
    items = {'apple', 'samsung'}
    return templates.TemplateResponse("index.html", {"request": request, "items": items})
# uvicorn main:app --reload
```

<img src='../source/img/09/06/02.png' width='1000px' height='600px'>

### **7. Comments in Jinja2**
Comments provide useful in-code documentation without appearing in the final rendered page.

**Comment Syntax:** `{# This is a comment #}`
```html
{# This is a comment and will not be visible in the browser #}
```

### **Conclusion**
Combining **FastAPI** with **Jinja2** offers a powerful solution for creating dynamic web pages with efficient data management. By leveraging features such as template inheritance, filters, and control statements, developers can build well-structured, maintainable, and scalable web applications.

If implemented effectively, this approach enhances both development efficiency and the user experience.